# TRACLUS

In [14]:
import math
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd

In [15]:
data_dir = '../data/porto-data'
data_name = "small-porto"

In [16]:
trajectories = pd.read_csv("{}/{}.csv".format(data_dir, data_name), header=0, index_col="TRIP_ID")
total_traj_num = len(trajectories)

In [17]:
total_traj_num

100000

In [18]:
trajectories

,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
TRIP_ID,,,,,,,,
1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...
1374433338620000398,B,NaN,36.0,20000398,1374433338,A,False,"[[-8.649297,41.154309],[-8.650044,41.154228],[..."
1374433379620000506,B,NaN,53.0,20000506,1374433379,A,False,"[[-8.6139,41.141133],[-8.613891,41.141115],[-8..."
1374433756620000435,B,NaN,9.0,20000435,1374433756,A,False,"[[-8.606475,41.144508],[-8.60652,41.144517],[-..."


In [19]:
shortest, longest = 20, 1200

processed_trajectories = []
for i, (idx, traj) in enumerate(trajectories.iterrows()):
    if i % 10000 == 0:
        print("Complete: {}; Total: {}".format(i, total_traj_num))
    polyline = eval(traj["POLYLINE"])
    coordinates = []
    if shortest <= len(polyline) <= longest:
        for lng, lat in polyline:
            coordinates.append([lng, lat])
        processed_trajectories.append(coordinates)

print("Valid trajectory num:", len(processed_trajectories))

Complete: 0; Total: 100000
Complete: 10000; Total: 100000
Complete: 20000; Total: 100000
Complete: 30000; Total: 100000
Complete: 40000; Total: 100000
Complete: 50000; Total: 100000
Complete: 60000; Total: 100000
Complete: 70000; Total: 100000
Complete: 80000; Total: 100000
Complete: 90000; Total: 100000
Valid trajectory num: 89011


In [20]:
fout = open("{}/processed_{}.csv".format(data_dir, data_name), 'w')
fout.write("POLYLINE\n")
for traj in processed_trajectories:
    fout.write('"' + str(traj) + '"' + "\n")
fout.close()

# Show trajectory

In [21]:
for i, (idx, traj) in enumerate(trajectories.iterrows()):
    lng = []
    lat = []
    if i == 0:
        for coordinate in eval(traj["POLYLINE"]):
            lng.append(coordinate[0])
            lat.append(coordinate[1])
            plt.plot(lng, lat, linewidth=5, color="blue")
            # plt.scatter(lng, lat, color="blue")
        break
mplleaflet.show()

In [22]:
test_traj = trajectories.iloc[0]
test_traj

CALL_TYPE                                                       C
ORIGIN_CALL                                                   NaN
ORIGIN_STAND                                                  NaN
TAXI_ID                                                  20000589
TIMESTAMP                                              1372636858
DAY_TYPE                                                        A
MISSING_DATA                                                False
POLYLINE        [[-8.618643,41.141412],[-8.618499,41.141376],[...
Name: 1372636858620000589, dtype: object

# Partition

In [23]:
def find_longer_shorter(line1, line2):
    line1_length = segment_length(line1)
    line2_length = segment_length(line2)
    if line1_length > line2_length:
        return line1, line2
    else:
        return line2, line1


def segment_length(line):
    length = ((line[0][0] - line[1][0])**2 + (line[0][1] - line[1][1])**2)**0.5
    return length


def get_unit_vector(line):
    line_length = segment_length(line)
    if line_length == 0:
        return [0, 0]
    unit_vector = [line[1][0], line[1][1]]
    unit_vector[0] = (unit_vector[0] - line[0][0]) / line_length
    unit_vector[1] = (unit_vector[1] - line[0][1]) / line_length
    return unit_vector


def projection_distance(point, line):
    diff_x = point[0] - line[0][0]
    diff_y = point[1] - line[0][1]
    unit_vector = get_unit_vector(line)
    projection_distance =  abs(diff_x * unit_vector[1] - diff_y * unit_vector[0])
    return projection_distance


def perpendicular_distance(line1, line2):
    longer, shorter = find_longer_shorter(line1, line2)
    distance1 = projection_distance(shorter[0], longer)
    distance2 = projection_distance(shorter[1], longer)
    if distance1 == 0 and distance2 == 0:
        return 0
    perpendicular_distance = (distance1**2 + distance2**2) / (distance1 + distance2)
    return round(perpendicular_distance, 10)


def angle_distance(line1, line2):
    longer, shorter = find_longer_shorter(line1, line2)
    unit_vector_shorter = get_unit_vector(shorter)
    unit_vector_longer = get_unit_vector(longer)
    sin_coefficient = unit_vector_shorter[0] * unit_vector_longer[1] - unit_vector_shorter[1] * unit_vector_longer[0]
    angle_distance = sin_coefficient * segment_length(shorter)
    return abs(angle_distance)


def cost(trajectory, start_index, current_index, partition):
    if partition:
        segment = [trajectory[start_index], trajectory[current_index]]
        seg_length = segment_length(segment)
        if seg_length != 0:
            LH = math.log2(segment_length(segment))
        else:
            LH = 0
        LDH_p = 0
        LDH_a = 0
        i = 0
        while start_index + i < current_index:
            index1 = start_index + i
            index2 = start_index + i + 1
            line = [trajectory[index1], trajectory[index2]]
            distance_p = perpendicular_distance(line, segment)
            distance_a = angle_distance(line, segment)
            LDH_p += distance_p
            LDH_a += distance_a
            i += 1
        if LDH_p == 0 and LDH_a == 0:
            LDH = 0
        elif LDH_p == 0 and LDH_a != 0:
            LDH = math.log2(LDH_a)
        elif LDH_p != 0 and LDH_a == 0:
            LDH = math.log2(LDH_p)
        else:
            LDH = math.log2(LDH_p) + math.log2(LDH_a)
        cost = LH + LDH
    else:
        cost = 0
        i = 0
        while start_index + i < current_index:
            index1 = start_index + i
            index2 = start_index + i + 1
            line = [trajectory[index1], trajectory[index2]]
            line_length = segment_length(line)
            if line_length != 0:
                cost += math.log2(line_length)
            i += 1
    return cost

def trajectory_partition(trajectory):
    cp = []
    cp_index = []
    cp.append(trajectory[0])
    cp_index.append(0)
    start_index = 0
    length = 1
    c = 0
    while start_index + length < len(trajectory):
        current_index = start_index + length
        cost_par = cost(trajectory, start_index, current_index, True)
        cost_non = cost(trajectory, start_index, current_index, False)
        if cost_par > cost_non:
            cp.append(trajectory[current_index-1])
            cp_index.append(current_index-1)
            start_index = current_index-1
            length = 1
        else:
            length += 1
        c += 1
    cp.append(trajectory[-1])
    cp_index.append(len(trajectory)-1)
    return cp, cp_index

In [24]:
test_traj_coord = eval(test_traj["POLYLINE"])
cp, cp_index = trajectory_partition(test_traj_coord)

In [25]:
# display partition result
lng = []
lat = []
for coordinate in cp:
    lng.append(coordinate[0])
    lat.append(coordinate[1])
    plt.plot(lng, lat, linewidth=5, color="blue")
mplleaflet.show()

In [26]:
cp_index

[0, 3, 6, 8, 11, 13, 15, 17, 19, 20, 22]

# All trajectories

In [27]:
trajectories = pd.read_csv("../data/porto-data/processed_small-porto.csv")
trajectories

,POLYLINE
0,"[[-8.618643, 41.141412], [-8.618499, 41.141376..."
1,"[[-8.612964, 41.140359], [-8.613378, 41.14035]..."
2,"[[-8.574678, 41.151951], [-8.574705, 41.151942..."
3,"[[-8.645994, 41.18049], [-8.645949, 41.180517]..."
4,"[[-8.615502, 41.140674], [-8.614854, 41.140926..."
...,...
89006,"[[-8.649297, 41.154309], [-8.650044, 41.154228..."
89007,"[[-8.6139, 41.141133], [-8.613891, 41.141115],..."
89008,"[[-8.606475, 41.144508], [-8.60652, 41.144517]..."
89009,"[[-8.604945, 41.149692], [-8.605368, 41.149773..."


In [28]:
all_cp_index = []
for i, (idx, traj) in enumerate(trajectories.iterrows()):
    if i % 10000 == 0:
        print("progress:", i, "/", len(trajectories))
    traj_coord = eval(traj["POLYLINE"])
    _, cp_index = trajectory_partition(traj_coord)
    all_cp_index.append(cp_index)
print("finished")

progress: 0 / 89011
progress: 10000 / 89011
progress: 20000 / 89011
progress: 30000 / 89011
progress: 40000 / 89011
progress: 50000 / 89011
progress: 60000 / 89011
progress: 70000 / 89011
progress: 80000 / 89011
finished


In [29]:
trajectories["CP_INDEX"] = all_cp_index
trajectories

,POLYLINE,CP_INDEX
0,"[[-8.618643, 41.141412], [-8.618499, 41.141376...","[0, 3, 6, 8, 11, 13, 15, 17, 19, 20, 22]"
1,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...","[0, 3, 6, 9, 11, 13, 16, 18, 21, 24, 27, 29, 3..."
2,"[[-8.574678, 41.151951], [-8.574705, 41.151942...","[0, 3, 6, 8, 10, 12, 14, 16, 18, 21, 23, 26, 2..."
3,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...","[0, 2, 4, 6, 8, 10, 12, 15, 17, 19, 21, 24, 26..."
4,"[[-8.615502, 41.140674], [-8.614854, 41.140926...","[0, 2, 5, 8, 11, 14, 16, 18, 20, 22, 25]"
...,...,...
89006,"[[-8.649297, 41.154309], [-8.650044, 41.154228...","[0, 2, 5, 8, 11, 14, 16, 19, 21, 23, 26, 28, 3..."
89007,"[[-8.6139, 41.141133], [-8.613891, 41.141115],...","[0, 3, 5, 8, 11, 13, 15, 17, 20, 23, 26, 29, 3..."
89008,"[[-8.606475, 41.144508], [-8.60652, 41.144517]...","[0, 3, 5, 7, 9, 11, 13, 16, 19, 22, 24, 26, 29..."
89009,"[[-8.604945, 41.149692], [-8.605368, 41.149773...","[0, 2, 4, 6, 8, 10, 13, 15, 18, 22, 24, 26, 28..."


In [30]:
fout = open("../data/porto-data/processed_small_porto_with_cp.csv", 'w')
fout.write("POLYLINE, CP_INDEX\n")
for i, (idx, traj) in enumerate(trajectories.iterrows()):
    fout.write('"' + str(traj["POLYLINE"]) + '",')
    fout.write('"' + str(traj["CP_INDEX"]) + '"\n')
fout.close()